# My attempts to save datasets into zarrs in the google bucket. 
I am trying it with bedmachine and with PISM output.

One problem was with the bedmachine dataset. I sorted that and then managed to get a small dataset (initially loaded from a netcdf) into and back out of the google bucket. Now the issue that I am running out of memory and the server crashes. 

Next step is to try to get it running on the cluster, but .to_zarr does not run on the cluster as it stands. 

In [1]:
import dask
import dask.array as da
import dask.delayed
from dask.distributed import Client
import dask_gateway
import numpy as np
import xarray as xr
xr.set_options(display_style="html")
import fsspec
import gcsfs

### 1. Start a cluster

In [2]:
# get the dask-gateway version
dask_gateway.__version__
# show the default dask-gateway settings
dask.config.config['gateway']
#default gateway call
gateway = dask_gateway.Gateway()
# default new_cluster call
cluster = gateway.new_cluster()
#gateway = Gateway()
gateway.list_clusters()
# the dashboard_link property will show the link that can be pasted into the Dask labextension
cluster.dashboard_link
# scale cluster to 8 workers using the scale() method

In [31]:
cluster.scale(4)
# connect a client
# the distributed client is used for running parallel tasks with Dask
client = Client(cluster)
cluster

### 2a. load the netcdf
This, I now understand, does not load a dask array, just a lazily loaded xarray.

In [16]:
url = 'https://storage.googleapis.com/ldeo-glaciology/bedmachine/BedMachineAntarctica_2019-11-05_v01.nc'  
with  fsspec.open(url, mode='rb')  as openfile:  
    bm = xr.open_dataset(openfile)  

# remove the variable mapping because it was causing an error in the write to zarr
bm.attrs = {**bm.attrs, **bm.mapping.attrs} # but keep the information in the attributes of the whole dataset. 
bm = bm.drop('mapping')   # remove the variable. 
type(bm.thickness.data)

numpy.ndarray

### 2b. load the netcdf
By defining chunk size, this one loads a dask array.

In [7]:
import gcsfs
import xarray as xr

# the four line below come from https://github.com/pangeo-data/pangeo-datastore/issues/116#issuecomment-729928936
gcs = gcsfs.GCSFileSystem()
url = 'gs://ldeo-glaciology/bedmachine/BedMachineAntarctica_2019-11-05_v01.nc'  
openfile = gcs.open(url, mode='rb') 
bm_dask = xr.open_dataset(openfile, chunks=3000)

# remove the variable mapping because it was causing an error in the write to zarr
bm_dask.attrs = {**bm_dask.attrs, **bm_dask.mapping.attrs} # but keep the information in the attributes of the whole dataset. 
bm_dask = bm_dask.drop('mapping')   # remove the variable. 
#type(bm_dask.thickness.data)
#bm_dask = bm_dask.isel(x=slice(0,13300), y = slice(0, 13300))
bm_dask

<xarray.Dataset>
Dimensions:    (x: 13300, y: 13300)
Coordinates:
  * x          (x) int32 -3333000 -3332500 -3332000 ... 3315500 3316000 3316500
  * y          (y) int32 3333000 3332500 3332000 ... -3315500 -3316000 -3316500
Data variables:
    mask       (y, x) int8 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    firn       (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    surface    (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    thickness  (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    bed        (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    errbed     (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    source     (y, x) int8 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    geoid      (y, x) int16 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
Attributes:
    Conventions:                            CF-1.7
    Title:                                  BedMachine Antarctica
    Author:                                 Mathieu Morlighem
    version:                                05-Nov-2019 (v1.38)
    nx:                                     [13333.]
    ny:                                     [13333.]
    Projection:                             Polar Stereographic South (71S,0E)
    proj4:                                  +init=epsg:3031
    sea_water_density (kg m-3):             [1027.]
    ice_density (kg m-3):                   [917.]
    xmin:                                   [-3333000]
    ymax:                                   [3333000]
    spacing:                                [500]
    no_data:                                [-9999.]
    license:                                No restrictions on access or use
    Data_citation:                          Morlighem M. et al., (2019), Deep...
    Notes:                                  Data processed at the Department ...
    grid_mapping_name:                      polar_stereographic
    latitude_of_projection_origin:          [-90.]
    standard_parallel:                      [-71.]
    straight_vertical_longitude_from_pole:  [0.]
    semi_major_axis:                        [6378273.]
    inverse_flattening:                     [298.27940504]
    false_easting:                          [0.]
    false_northing:                         [0.]

In [9]:
%%time
bm_dask.thickness.mean().compute()

CPU times: user 37.9 ms, sys: 10.7 ms, total: 48.6 ms
Wall time: 2.56 s


<xarray.DataArray 'thickness' ()>
array(589.04956, dtype=float32)

### 3. take small subsets of the data (from the non-dask and dask arrays)

In [17]:
bm_small = bm.isel(x=slice(0,20), y = slice(0, 20))  # create a very small version of the dataset for testing the upload and download
bm_small.nbytes/1e3   #  it is only 10 MB

9.76

In [13]:
bm_dask_small = bm_dask.isel(x=slice(0,20), y = slice(0, 20))  # create a very small version of the dataset for testing the upload and download
bm_dask_small#.nbytes/1e3   #  it is only 10 MB

<xarray.Dataset>
Dimensions:    (x: 20, y: 20)
Coordinates:
  * x          (x) int32 -3333000 -3332500 -3332000 ... -3324000 -3323500
  * y          (y) int32 3333000 3332500 3332000 ... 3324500 3324000 3323500
Data variables:
    mask       (y, x) int8 dask.array<chunksize=(20, 20), meta=np.ndarray>
    firn       (y, x) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
    surface    (y, x) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
    thickness  (y, x) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
    bed        (y, x) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
    errbed     (y, x) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
    source     (y, x) int8 dask.array<chunksize=(20, 20), meta=np.ndarray>
    geoid      (y, x) int16 dask.array<chunksize=(20, 20), meta=np.ndarray>
Attributes:
    Conventions:                            CF-1.7
    Title:                                  BedMachine Antarctica
    Author:                                 Mathieu Morlighem
    version:                                05-Nov-2019 (v1.38)
    nx:                                     [13333.]
    ny:                                     [13333.]
    Projection:                             Polar Stereographic South (71S,0E)
    proj4:                                  +init=epsg:3031
    sea_water_density (kg m-3):             [1027.]
    ice_density (kg m-3):                   [917.]
    xmin:                                   [-3333000]
    ymax:                                   [3333000]
    spacing:                                [500]
    no_data:                                [-9999.]
    license:                                No restrictions on access or use
    Data_citation:                          Morlighem M. et al., (2019), Deep...
    Notes:                                  Data processed at the Department ...
    grid_mapping_name:                      polar_stereographic
    latitude_of_projection_origin:          [-90.]
    standard_parallel:                      [-71.]
    straight_vertical_longitude_from_pole:  [0.]
    semi_major_axis:                        [6378273.]
    inverse_flattening:                     [298.27940504]
    false_easting:                          [0.]
    false_northing:                         [0.]

### 4. write the small subset to the bucket
bm_small (the non-dask array) can be written to a zarr directory in our bucket no problem. 

In [15]:
bm_small_mapper = fsspec.get_mapper('gs://ldeo-glaciology/temp/bm_small9.zarr', mode='ab',
                            token='../secrets/ldeo-glaciology-bc97b12df06b.json')  # get a mapper object using the token stored in the ooi environment
bm_small.to_zarr(bm_small_mapper, mode='w');   # write the dataset to zarr in the google bucket

The next cell loads the small subset as a dask array, but fails by killing a worker

In [20]:
#bm_small_mapper = fsspec.get_mapper('gs://ldeo-glaciology/temp/bm_small4.zarr') # This also works - just to make sure we dont need the token to access
#bm_small_reloaded = xr.open_zarr(bm_small_mapper) # reload the dataset using the same mapper as before
bm_small_reloaded.load()#.identical(bm_small)    # check that what we get back is the same as what we tried to load up.

KilledWorker: ('zarr-a39495395d9156150ded4ba45029c835', <Worker 'tls://10.36.56.30:39121', name: dask-worker-8ea1f5b7cde44877a9fe9afe5cac90cd-gpfs6, memory: 0, processing: 4>)

### Next write the small set of the dask array to the bucket. 
The top half of this cell I got from https://github.com/pangeo-data/pangeo-datastore/issues/116#issuecomment-729357633
it apparently provides the workers access to the token

In [14]:
import json
with open('../secrets/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)
gcs = gcsfs.GCSFileSystem(token=token)

bm_small_mapper = gcs.get_mapper('gs://ldeo-glaciology/temp/bm_small_dask_1.zarr')
bm_dask_small.to_zarr(bm_small_mapper, mode='w');

### Writing the non-dask version of the full dataset to zarr fails on the largest version of ooi.pangeo.io, but on the larger https://us-central1-b.gcp.pangeo.io/ it works. 

In [23]:
bm_dask

<xarray.Dataset>
Dimensions:    (x: 13300, y: 13300)
Coordinates:
  * x          (x) int32 -3333000 -3332500 -3332000 ... 3315500 3316000 3316500
  * y          (y) int32 3333000 3332500 3332000 ... -3315500 -3316000 -3316500
Data variables:
    mask       (y, x) int8 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    firn       (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    surface    (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    thickness  (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    bed        (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    errbed     (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    source     (y, x) int8 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    geoid      (y, x) int16 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
Attributes:
    Conventions:                            CF-1.7
    Title:                                  BedMachine Antarctica
    Author:                                 Mathieu Morlighem
    version:                                05-Nov-2019 (v1.38)
    nx:                                     [13333.]
    ny:                                     [13333.]
    Projection:                             Polar Stereographic South (71S,0E)
    proj4:                                  +init=epsg:3031
    sea_water_density (kg m-3):             [1027.]
    ice_density (kg m-3):                   [917.]
    xmin:                                   [-3333000]
    ymax:                                   [3333000]
    spacing:                                [500]
    no_data:                                [-9999.]
    license:                                No restrictions on access or use
    Data_citation:                          Morlighem M. et al., (2019), Deep...
    Notes:                                  Data processed at the Department ...
    grid_mapping_name:                      polar_stereographic
    latitude_of_projection_origin:          [-90.]
    standard_parallel:                      [-71.]
    straight_vertical_longitude_from_pole:  [0.]
    semi_major_axis:                        [6378273.]
    inverse_flattening:                     [298.27940504]
    false_easting:                          [0.]
    false_northing:                         [0.]

In [30]:
#bm_mapper = fsspec.get_mapper('gs://ldeo-glaciology/bedmachine/bm.zarr', mode='ab',
#                            token='../secrets/ldeo-glaciology-bc97b12df06b.json')

bm_mapper = gcs.get_mapper('gs://ldeo-glaciology/bedmachine/bm4.zarr')
bm_dask.to_zarr(bm_mapper, mode='w');

In [25]:
bm_reloaded = xr.open_zarr(bm_mapper)  


False

In [35]:
bm_reloaded.identical(bm_dask)

KeyboardInterrupt: 

In [15]:
cluster.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
